## Evaluate PF-controller neural network

**Note:** This evaluation follows a leave-one-unit-out protocol applied independently to each performance metric. For a given evaluation unit and performance metric, the particle filter and its neural transition model are inferred using data from all remaining units.

In [55]:
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from src.models.particle_filter import ParticleFilterMLP, ParticleFilterModel
from src.models.normal import NormalDegradationModel as NModel
import imageio
import matplotlib.pyplot as plt

from src.helpers.visualization import create_pf_prediciton_frame
from src.helpers.seed import set_global_seed

In [56]:
data_name = "DS05"
perform_name = "SmLPC"
SEED = 42

pf_dir_name = 'pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05'


experiment_dir = Path('experiments')/data_name
pf_dir = experiment_dir/ pf_dir_name/perform_name
states_dir = experiment_dir/'states'/ perform_name

set_global_seed(SEED)

## Hyper Parameters

In [57]:
n_particles = 1800
multiply_scale = 0.05
start_idx = 1

net = ParticleFilterMLP(state_dim=NModel.state_dim(),hidden_dims=[128, 128, 64,32],activation=lambda: nn.LeakyReLU(0.05))

## Import development data

In [58]:
hi_df = pd.read_csv(experiment_dir/'hidata_dev.csv')
del hi_df['hs']
units = hi_df['unit'].astype(int).unique().tolist()
units 

[1, 2, 3, 4, 5, 6]

In [59]:
perform_names = [col for col in hi_df.columns if col not in ['unit','cycle']]

performs = {name: 
    {unit: hi_df[hi_df['unit']==unit][name].values for unit in units} 
    for name in perform_names
}
time = {unit: hi_df[hi_df['unit']==unit]['cycle'].values for unit in units}

## Prepare data

In [60]:
eval_data = {}
for unit in units:
    t_data = time[unit]
    s_data = performs[perform_name][unit]
    eval_data[unit]=torch.tensor(np.stack([t_data, s_data],axis=1),dtype=torch.float32)


## Load component (base) models

In [61]:
train_degmodels = {}
for eval_unit in units:
    degmodels=[]
    for unit, perform in performs[perform_name].items():
        if unit == eval_unit:
            continue
        best_model = NModel()
        best_model.load_state_dict(
            torch.load(states_dir /f'unit_{unit}'/ "best_model.pt")
        )
        degmodels.append(best_model)
    train_degmodels[eval_unit] = degmodels
    

## Load PF-net

In [62]:
ckpt = torch.load(pf_dir/'checkpoint.pt')
net.load_state_dict(ckpt['model_state'])

net.eval()  

ParticleFilterMLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.05)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.05)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): LeakyReLU(negative_slope=0.05)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): LeakyReLU(negative_slope=0.05)
    (8): Linear(in_features=32, out_features=16, bias=True)
  )
)

## Run Particle Filter with (leraned net)

Save video

In [63]:
t_grid = np.linspace(0.1, 100, 80)
s_grid = np.linspace(0.0, 1.0, 60)
conf_level = 0.95

for eval_unit in units:
    degmodels = train_degmodels[eval_unit]
    t_data = eval_data[eval_unit][:, 0]
    s_data = eval_data[eval_unit][:, 1]
    t_data_np = t_data.cpu().numpy()
    s_data_np = s_data.cpu().numpy()

    pf = ParticleFilterModel(
        base_models=degmodels,
        net=net,
        n_particles=n_particles,
        multiply_scale=multiply_scale,
        name=perform_name,
    ).eval()

    frames = []
    for k in range(start_idx, len(t_data)):
        pf.step(
            t_obs=t_data[:k],
            s_obs=s_data[:k],
        )
        
        lower, mean, upper = pf.mixture.uncertainty_interval(s=torch.zeros(1), level=conf_level)
        # --- render frame ---
        fig, ax = plt.subplots(figsize=(10, 8))
        
        create_pf_prediciton_frame(
            ax,
            pf,
            t_grid,
            s_grid,
            t_data_np,
            s_data_np,
            pred_interval=(lower.item(), mean.item(), upper.item()),
            conf_level=conf_level,
            current_step = k,
            dist_vmax=0.25,
        )
        # create frame
        fig.canvas.draw()
        frame = np.asarray(fig.canvas.renderer.buffer_rgba())
        plt.close(fig)
        frames.append(frame)
  
    ## Save video
    video_path = pf_dir / f"pf_eval{eval_unit}.mp4"

    with imageio.get_writer(video_path, fps=8, macro_block_size=1) as writer:
        for frame in frames:
            writer.append_data(frame)

    print(f"🎬 Video saved to {video_path}")


🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval1.mp4
🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval2.mp4
🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval3.mp4
🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval4.mp4
🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval5.mp4
🎬 Video saved to experiments/DS05/pf_perform_normal_multnoise0.05_net128x128x64x32leaky0.05/SmLPC/pf_eval6.mp4
